In [3]:
import numpy as np
import pandas as pd
import os
import torch
import math
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
import torch.nn as nn

import matplotlib.pyplot as plt
# %matplotlib inline

# import wandb

# import imageio


Bad key "text.kerning_factor" on line 4 in
C:\Users\levik\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [4]:
import os.path

def build_path(path):
    for i in path:
        if not os.path.exists(i):
            os.makedirs(i)


In [6]:
# load the dataset

imax = 381294

skiprows = 2
max_rows = 1000
goal_rows = 50000-5000

x = np.loadtxt("generate/new_algebra_input_100000.txt",delimiter=",",max_rows=2)

for i in range(int(goal_rows/max_rows)):
    test = np.loadtxt("generate/new_algebra_input_100000.txt",delimiter=",",max_rows=max_rows,skiprows=int(1+i*max_rows))
    x = np.concatenate((x,test))
test=0

    
y = np.loadtxt("generate/new_algebra_output_100000.txt",delimiter=",",max_rows=2*2)

for i in range(int((goal_rows/max_rows))):
#     print(max_rows)
#     print(int(3+i*max_rows))
#     print(i,(1+i*max_rows)*2)
    test1 = np.loadtxt("generate/new_algebra_output_100000.txt",delimiter=",",max_rows=max_rows,skiprows=int(1+i*max_rows))
#     print(y.shape,test.shape)
    y = np.concatenate((y,test1))

print(x.shape,y.shape)


(45002, 3041) (45004, 186)


In [5]:
from torch.utils.data.sampler import SubsetRandomSampler

# split into train and test

BATCH_SIZE = 64
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(x)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]

# # print(train_indices)
# # print(val_indices)
# # print(len(train_indices), len(val_indices))
# train_sampler = SubsetRandomSampler(train_indices)
# valid_sampler = SubsetRandomSampler(val_indices)

NameError: name 'x' is not defined

In [105]:
# normalize

Xmean, Xstd = x.mean(axis=0), x.std(axis=0)
Ymean, Ystd = y.mean(axis=0), y.std(axis=0)

# print(Xmean, Xstd)
# print(Ymean, Ystd)


for i in range(Xstd.size):
    if (Xstd[i]==0):
        Xstd[i]=1
for i in range(Ystd.size):
    if (Ystd[i]==0):
        Ystd[i]=1
        
        
x = (x - Xmean) / Xstd
y = (y - Ymean) / Ystd

In [106]:
# Save the normalization values
# np.savetxt("Xmean_algebra_100000_11.txt",Xmean,delimiter=",",fmt="%1.10f")
# np.savetxt("Xstd_algebra_100000_11.txt",Xstd,delimiter=",",fmt="%1.10f")
# np.savetxt("Ymean_algebra_100000_11.txt",Ymean,delimiter=",",fmt="%1.10f")
# np.savetxt("Xstd_algebra_100000_11.txt",Ystd,delimiter=",",fmt="%1.10f")



In [107]:
# actually split the dataset
xtrain = x[train_indices]
ytrain = y[train_indices]

xtest = x[test_indices]
ytest = y[test_indices]

Xdim = xtrain.shape[1]
Ydim = ytrain.shape[1]

In [108]:
# Define the network tensors

x = torch.tensor(xtrain)
y = torch.tensor(ytrain)

testx = torch.tensor(xtest)
testy = torch.tensor(ytest)

In [110]:
print(x.shape)

torch.Size([12002, 3041])


In [111]:
print(y.shape)

torch.Size([12002, 186])


In [113]:
# weight initializations

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.normal_(m.bias, 0.0, 0.001)
        torch.nn.init.zeros_(m.weight)

# tat = nn.Sequential(nn.Linear(2, 2), nn.Linear(2, 2))
# tat.apply(init_weights)

In [121]:
# Define the torch network

x, y = Variable(x), Variable(y)
testx, testy = Variable(testx), Variable(testy)

hidden_size = 512
dropout_p = 0.3
hidden_layers = 2
activation = "ELU"


net = torch.nn.Sequential(
        
        torch.nn.LayerNorm(Xdim),
        torch.nn.Dropout(p=dropout_p),

    
        torch.nn.Linear(Xdim, hidden_size),
        torch.nn.LayerNorm(hidden_size),
    
        torch.nn.ELU(),
        torch.nn.Dropout(p=dropout_p),
    
        torch.nn.Linear(hidden_size, hidden_size),
        torch.nn.LayerNorm(hidden_size),
    
        torch.nn.ELU(),
        torch.nn.Dropout(p=dropout_p),
    
    
#         torch.nn.Linear(hidden_size, hidden_size),
#         torch.nn.LayerNorm(hidden_size),

#         torch.nn.ELU(),
#         torch.nn.Dropout(p=dropout_p),
        
        torch.nn.Linear(hidden_size, Ydim),
    )


# BATCH_SIZE = 64
epochs = 20
learning_rate = 0.01
weight_decay = 0.025
momentum = 0.9
optim = "SGD"

net.apply(init_weights)

# optimizer = torch.optim.AdamW(net.parameters(), lr=learning_rate, weight_decay=weight_decay)
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

def cross_entropy(Y, H3):
    return 
loss_func = nn.MSELoss()



torch_dataset = Data.TensorDataset(x,y)
test_dataset = Data.TensorDataset(x,y)

loader = Data.DataLoader(
    dataset=torch_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, num_workers=2,)

test_loader = Data.DataLoader(
    dataset=torch_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, num_workers=2,)

# train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
#                                            sampler=train_sampler)
# validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
#                                                 sampler=valid_sampler)

In [122]:
# wandb for tracking the training process
import wandb


In [123]:
wandb.login()

True

In [124]:
# %env WANDB_SILENT=true

In [ ]:
# %%wandb

# fig, ax = plt.subplots(figsize=(16,10))
wandb.init(project="pytorch_motions_algebra", config={"learning-rate": learning_rate, "epochs":epochs, 
                                                      "batch-size": BATCH_SIZE, "weight-decay":weight_decay,
                                                      "hidden-size":hidden_size, "dropout-p":dropout_p,
                                                      "hidden-layers":hidden_layers,"activation":activation,
                                                      "random-seed":random_seed, "validation-split":validation_split,
                                                      "dataset-size":dataset_size, "optimizer":optim
                                                     }, reinit=True)

# Training process

for epoch in range(epochs):
    train_loss = 0
    test_loss = 0
    train_steps = 0
    test_steps = 0
    net.train()
    for step, (batch_x, batch_y) in enumerate(loader):
        b_x = Variable(batch_x)
        b_y = Variable(batch_y).float()

        prediction = net(b_x.float())
        
#         print(prediction.shape,b_y.shape)
        
        loss = loss_func(prediction, b_y)
        
        train_loss = train_loss + loss
        train_steps = step
        
        optimizer.zero_grad()   # clear gradients for next train
        loss.backward()         # backpropagation, compute gradients
        optimizer.step() 
#     print(loss.item())
        
        
    train_loss = train_loss / train_steps
    wandb.log({"train_loss": train_loss})
    
    net.eval()
    for step, (batch_x, batch_y) in enumerate(test_loader):
        b_x = Variable(batch_x)
        b_y = Variable(batch_y).float()

        prediction = net(b_x.float())
        
#         print(prediction.shape,b_y.shape)
        
        loss = loss_func(prediction, b_y)
        
        test_loss = test_loss + loss
        test_steps = step

    test_loss = test_loss / test_steps
    wandb.log({"test_loss": test_loss})
    
    print('Epoch:',  '%04d' % (epoch + 1), 'trainingloss =', train_loss)
    print('Epoch:', '%04d' % (epoch + 1), 'testloss =', test_loss)
        
print('Done!')
wandb.finish()

In [ ]:
# Save the network

In [1]:
path = "model_agebra_100000.pt"
torch.save(net.state_dict(), path)


NameError: name 'torch' is not defined

In [17]:
# Loading the network

hidden_size = 1024
dropout_p = 0.3
hidden_layers = 3
activation = "ELU"
# BATCH_SIZE = 64
epochs = 20
learning_rate = 0.01
weight_decay = 0.025
momentum = 0.9
optim = "SGD"


path = "model_algebra_100000.pt"


In [18]:
model = torch.nn.Sequential(
        
        torch.nn.LayerNorm(Xdim,
        torch.nn.Dropout(p=dropout_p),

    
        torch.nn.Linear(Xdim, hidden_size),
        torch.nn.LayerNorm(hidden_size),
    
        torch.nn.ELU(),
        torch.nn.Dropout(p=dropout_p),
    
        torch.nn.Linear(hidden_size, hidden_size),
        torch.nn.LayerNorm(hidden_size),
    
        torch.nn.ELU(),
        torch.nn.Dropout(p=dropout_p),
    
    
        torch.nn.Linear(hidden_size, hidden_size),
        torch.nn.LayerNorm(hidden_size),

        torch.nn.ELU(),
        torch.nn.Dropout(p=dropout_p),
        
        torch.nn.Linear(hidden_size, Ydim),
    )


model.load_state_dict(torch.load(path))


<All keys matched successfully>

In [19]:
# Comparing validation data

val_y = np.array(pd.read_csv("generate/algebra_output_100000_4.txt", header=None, skiprows=0))
val_x = np.array(pd.read_csv("generate/algebra_input_100000_4.txt", header=None, skiprows=0))

In [28]:
val_x = np.loadtxt("generate/new_algebra_input_100000.txt",delimiter=",",skiprows=29722,max_rows=501)
val_y = np.loadtxt("generate/new_algebra_output_100000.txt",delimiter=",",skiprows=29722,max_rows=501)

In [29]:
# val_y = np.array(pd.read_csv("algebra_output_0.txt", header=None, skiprows=0, nrows=600))
# val_x = np.array(pd.read_csv("algebra_input_0.txt", header=None, skiprows=0, nrows=600))
# print(val_y)

In [30]:
val_x = (val_x - Xmean) / Xstd
val_y = (val_y - Ymean) / Ystd

In [31]:
var_x = Variable(torch.tensor(np.array(val_x)))

prediction = model(var_x.float())

output = np.around(np.array((prediction.data * Ystd) + Ymean), 6)
val_y = np.around(np.array((val_y.data *Ystd) + Ymean), 6)

print(val_y.shape)
print(output.shape)

(501, 186)
(501, 186)


In [32]:
i = 10
j = 10

print(output[i, j:j+6])
print(val_y[i,j:j+6])

[-1.12000e-04 -7.40000e-05  5.34400e-02 -4.95010e-02  2.06380e-02
  1.32245e-01]
[ 0.        0.        0.055103 -0.052519  0.020292  0.110004]


In [33]:
print(val_y[i,j:j+6])

[ 0.        0.        0.055103 -0.052519  0.020292  0.110004]


In [25]:
print(output[0][0])

0.393918


In [34]:
with open("results/verybad_animation.txt", "w+") as f:
    for row in output:
        f.write(",".join(format(x, "1.6f") for x in row) + "\n")
        
with open("results/verygood_animation.txt", "w+") as f:
    for row in val_y:
        f.write(",".join(format(x, "1.6f") for x in row) + "\n")